In [198]:
import time 
import requests
from pprint import pprint
import csv
import pandas as pd
from config import key_rapid as api_key_rapid
from config2 import gkey as gkey_google



In [3]:
# Import list of magic towns 
magic_towns = pd.read_csv('Resources/pueblos_magicos.csv')
magic_towns_city = magic_towns['Ciudad'].head(5)
magic_towns_city

0      Aculco
1      Alamos
2     Amealco
3    Aquismon
4     Arteaga
Name: Ciudad, dtype: object

In [4]:
# Set up of lists 
lat = []
lon = []
name= []
destinationID = []

# Obtain basic data for each magic town: latitude, longitud, and destinationID 

url = "https://hotels4.p.rapidapi.com/locations/search"
headers = {
        'x-rapidapi-host': "hotels4.p.rapidapi.com",
        'x-rapidapi-key': api_key_rapid
        }

for city in magic_towns_city:
    
    querystring = {"locale":"en_MX","query":city}


    try:
        response = requests.request("GET", url, headers=headers, params=querystring)
        response_locations = response.json()
        lat.append(response_locations['suggestions'][0]['entities'][0]['latitude'])
        lon.append(response_locations['suggestions'][0]['entities'][0]['longitude'])
        name.append(response_locations['suggestions'][0]['entities'][0]['name'])
        destinationID.append(response_locations['suggestions'][0]['entities'][0]['destinationId'])  
        print(f'Proceso con exito  {city}')
        time.sleep(5)
    except:
        print(f'Ciudad no encontrada  {city}')
        


Proceso con exito  Aculco
Proceso con exito  Alamos
Proceso con exito  Amealco
Proceso con exito  Aquismon
Proceso con exito  Arteaga


In [34]:
#Data frame of the basic information per town
town_basic_data = pd.DataFrame({'Magic Town':name,'Latitude':lat,'Longitud':lon,'Destination Id':destinationID})

town_basic_data

,Magic Town,Latitude,Longitud,Destination Id
0,Aculco,20.089649,-99.759271,1904726
1,Alamos,27.051123,-108.876941,863503
2,Amealco,20.196826,-100.070282,851540
3,Aquismon,21.614999,-99.084884,857225
4,Arteaga,25.341357,-100.642369,830058


In [167]:
town_basic_data_id = town_basic_data.loc[:,'Destination Id']
town_basic_data_id

0    1904726
1     863503
2     851540
3     857225
4     830058
Name: Destination Id, dtype: object

In [173]:
# Lists to form the landmarks data frame  per magic town
town_id=[]
landmarks=[]
id_landmark = []
town_landmarks_data = []

#Lists to form the hotels data frame per magic town 
town_id_hotel=[]
hotel_name=[]
star_rating = []
price_per_night=[]
rooms_left=[]
total_guest_rev = []
rating_guest_rev=[]
locality=[]
hotel_type = []
id_hotel = []



In [174]:
# Get information for each magic town 
for det_id in town_basic_data_id:

    # Variables 
    page = 1
    processed_hotels = 0

    # API request page 1
    url = "https://hotels4.p.rapidapi.com/properties/list"

    querystring = {"currency":"MXN","locale":"en_MX","destinationId":det_id,"pageNumber":f'{page}',
                   "checkIn":"2020-10-04","checkOut":"2020-10-10","pageSize":"25","adults1":"2"}

    headers = {
        'x-rapidapi-host': "hotels4.p.rapidapi.com",
        'x-rapidapi-key': api_key_rapid
        }
    print(f'Page: {page}, Destination: {det_id}')
    
    response_details = requests.request("GET", url, headers=headers, params=querystring)
    response_details_json = response_details.json()

    #Obtain landmarks 
    
    for mark in response_details_json['data']['body']['filters']['landmarks']['items']:
        town_id.append(det_id)
        landmarks.append(mark['label'])
        id_landmark.append(mark['value'])
        

    #Obtain total of hotels for magic town
    total_hotels = response_details_json['data']['body']['searchResults']['totalCount']

    while total_hotels > processed_hotels:
        # Retrieve results starting from pag 2
        if processed_hotels > 0:
            page += 1
            
            print(f'Page: {page}, Destination: {det_id}')
            
            querystring = {"currency":"MXN","locale":"en_MX","destinationId":det_id,"pageNumber":f"{page}",
                       "checkIn":"2020-10-04","checkOut":"2020-10-10","pageSize":"25","adults1":"2"}
            
            response_details = requests.request("GET", url, headers=headers, params=querystring)
            response_details_json = response_details.json()

        #Add hotels to lists    
        for hotel in response_details_json['data']['body']['searchResults']['results']:
            # Do not add duplicates 
            if hotel['id'] not in id_hotel:
                town_id_hotel.append(det_id)
                hotel_name.append(hotel['name'])
                star_rating.append(hotel['starRating'])
                locality.append(hotel['address']['locality'])
                id_hotel.append(hotel['id'])

                try:
                    total_guest_rev.append(hotel['guestReviews']['total'])
                    rating_guest_rev.append(hotel['guestReviews']['rating'])
                except:
                    total_guest_rev.append(0)
                    rating_guest_rev.append('NA')

                try:
                    hotel_type.append(hotel['vrBadge'])
                except:
                    hotel_type.append('Hotel')

                try:
                    price_per_night.append(hotel['ratePlan']['price']['current'])
                except:
                    price_per_night.append(0)

                try:
                    rooms_left.append(hotel['roomsLeft'])
                except:
                    rooms_left.append(0)

            processed_hotels += 1

Page: 1, Destination: 1904726
Page: 2, Destination: 1904726
Page: 1, Destination: 863503
Page: 2, Destination: 863503
Page: 1, Destination: 851540
Page: 2, Destination: 851540
Page: 3, Destination: 851540
Page: 1, Destination: 857225
Page: 2, Destination: 857225
Page: 3, Destination: 857225
Page: 1, Destination: 830058
Page: 2, Destination: 830058
Page: 3, Destination: 830058


In [206]:
#Data frame of the landmarks
town_landmarks_data = pd.DataFrame({'Destination Id':town_id,'Landmarks':landmarks,'Landmark ID':id_landmark})

#Data farem of hotels per magic town 
town_hotels_data = pd.DataFrame({'Destination Id':town_id_hotel,'Hotel Name':hotel_name, 'Hotel ID':id_hotel,'Star rating':star_rating,
                                'Locality':locality,'Guest Reviews':total_guest_rev,'Guest Review Rating':rating_guest_rev,
                                'Hotel type':hotel_type,'Price per night':price_per_night,'Rooms left':rooms_left})

town_hotels_data

,Destination Id,Hotel Name,Hotel ID,Star rating,Locality,Guest Reviews,Guest Review Rating,Hotel type,Price per night,Rooms left
0,1904726,Collection O Xani Mui Hotel Boutique,572330,2.5,Aculco,0,NA,Hotel,MXN 533,1
1,1904726,Hotel Aculco,1623938048,2.5,Aculco,0,NA,Hotel,"MXN 1,652",0
2,1904726,Hotel La Esperanza,572268,2.5,Aculco,0,NA,Hotel,0,0
3,1904726,HOH Hotel,643875040,3.5,Aculco,11,7.6,Hotel,0,0
4,1904726,Mision La Muralla,236511,2.5,Amealco,149,8.8,Hotel,0,0
...,...,...,...,...,...,...,...,...,...,...
111,830058,City Express Saltillo Sur,341975,2.0,Saltillo,241,8.0,Hotel,0,0
112,830058,Fairfield Inn & Suites by Marriott Saltillo,373139,3.5,Ramos Arizpe,361,8.0,Hotel,0,0
113,830058,Hotel La Fuente,453471,2.5,Saltillo,68,7.2,Hotel,0,0
114,830058,Hotel Nuvó,475306,3.5,Saltillo,53,9.0,Hotel,0,0


In [179]:
town_hotels_data.to_csv('town_hotels.csv')

In [181]:
town_basic_data["Airport Name"] = ""
town_basic_data["Airport Address"]= ""
town_basic_data["Airport Rating"] = ""
town_basic_data

,Magic Town,Latitude,Longitud,Destination Id,Airport Name,Airport Address,Airport Rating
0,Aculco,20.089649,-99.759271,1904726,,,
1,Alamos,27.051123,-108.876941,863503,,,
2,Amealco,20.196826,-100.070282,851540,,,
3,Aquismon,21.614999,-99.084884,857225,,,
4,Arteaga,25.341357,-100.642369,830058,,,


In [200]:
# params dictionary to update each iteration
params = {
    "radius": 80000,
    "types": "airport",
    "keyword": "Comercial airport",
    "key": gkey_google
}

# Use the lat/lng we recovered to identify airports
for index, row in town_basic_data.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitud"]
    print(f'{lat}  ,  {lng}')
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        town_basic_data.loc[index, "Airport Name"] = name_address["results"][0]["name"]
        #cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        town_basic_data.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
        print(f"Airport found nearby. {index}")
    except (KeyError, IndexError):
        print(f"Missing field/result... skipping. {index}")

20.089649  ,  -99.759271
Missing field/result... skipping. 0
27.051123  ,  -108.876941
Airport found nearby. 1
20.196826  ,  -100.070282
Airport found nearby. 2
21.614999  ,  -99.084884
Airport found nearby. 3
25.341357  ,  -100.642369
Airport found nearby. 4


In [202]:
town_basic_data

,Magic Town,Latitude,Longitud,Destination Id,Airport Name,Airport Address,Airport Rating
0,Aculco,20.089649,-99.759271,1904726,,,
1,Alamos,27.051123,-108.876941,863503,Aeropuerto de Álamos,,0
2,Amealco,20.196826,-100.070282,851540,Querétaro Intercontinental Airport,,3.8
3,Aquismon,21.614999,-99.084884,857225,Tamuín National Airport,,4.2
4,Arteaga,25.341357,-100.642369,830058,"Aeropuerto Internacional de Saltillo ""Plan de ...",,4.1
